In [258]:
import numpy as np
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup as bs

### Here, the HTML file of the given URL will be extracted

In [259]:
with urllib.request.urlopen("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M") as website:
    url_file = website.read()
    html_file1 = url_file.decode("utf8")
    html_file2 = bs(html_file1,'lxml')

### Loop over the elements in the table, filling the Pandas Data Frame

In [260]:
data = pd.DataFrame({"PostalCode":[], 
                     "Borough":[],
                     "Neighbourhood":[]})
for element in html_file2.find_all('tr'):
    e = element.text.splitlines()
    if len(e) == 4:
        data_line = pd.DataFrame({"PostalCode":[e[1]], 
                                  "Borough":[e[2]],
                                  "Neighbourhood":[e[3]]})
        data = data.append(data_line)

### Cleaning the data (First row and 'Not assigned' borough rows deleted, 'Not assigned' neighbourhood = it's borough

In [261]:
# cleaning the data
data = data[data.PostalCode != 'Postcode']
data = data[data.Borough != 'Not assigned']
foo = lambda a: ", ".join(a)
data = data.groupby(['PostalCode'], as_index=False).agg({'Borough': 'min','Neighbourhood': foo}).reset_index(drop=True)
ind = data[data.Neighbourhood == 'Not assigned'].index.tolist()[0]
data.loc[ind]['Neighbourhood'] = data.loc[ind]['Borough']

In [262]:
data.shape

(103, 3)